In [20]:
import os
from datetime import datetime
from typing import List

import sys

sys.path.append(os.path.join('src'))

import numpy as np
import pandas as pd
from decode_mcd import DesignTargets, ContinuousTarget, CounterfactualsGenerator

from mcd_clip.datasets.combined_datasets import CombinedDataset
from mcd_clip.datasets.validations_lists import COMBINED_VALIDATION_FUNCTIONS
from mcd_clip.optimization.combined_optimizer import CombinedOptimizer, distance_column_name, TextEmbeddingTarget, \
    ImageEmbeddingTarget
from mcd_clip.optimization.embedding_similarity_optimizer import to_full_clips_dataframe
from mcd_clip.resource_utils import run_result_path, resource_path

import nest_asyncio
nest_asyncio.apply()

from mcd_clip.singletons import IMAGE_CONVERTOR



def render_some(full_df: pd.DataFrame, run_dir: str, batch_number: int, distance_column_suffix: str):
    batch_dir = os.path.join(run_dir, f"batch_{batch_number}_distance_{distance_column_suffix}")
    os.makedirs(batch_dir, exist_ok=True)
    clips = to_full_clips_dataframe(CombinedDataset(full_df).get_as_clips())
    images_paths = []
    for idx in clips.index:
        rendering_result = IMAGE_CONVERTOR.to_image(clips.loc[idx])
        image_path = os.path.join(batch_dir, f"bike_{idx}.svg")
        images_paths.append(image_path)
        with open(image_path, "wb") as file:
            file.write(rendering_result.image)

def _selective_sample(
        optimizer: CombinedOptimizer,
        generator: CounterfactualsGenerator,
        distance_column_index: int):
    desired_weight = 1
    objective_weights = [desired_weight * (1 - distance_column_index), desired_weight * distance_column_index]
    as_many = generator.sample_with_weights(num_samples=100, cfc_weight=0,
                                            gower_weight=0, avg_gower_weight=0,
                                            diversity_weight=0,
                                            bonus_objectives_weights=np.array(objective_weights).reshape((1, 2)),
                                            include_dataset=False)
    column_ = distance_column_name(distance_column_index)
    as_many[column_] = optimizer.predict(CombinedDataset(as_many))[column_]
    closes_cfs = as_many.sort_values(by=column_, ascending=True)[:4]
    print(f"Found closest cfs {closes_cfs[distance_column_name(distance_column_index)]}")
    return closes_cfs


def _balance_sample(generator: CounterfactualsGenerator,
                    objective_weights: List[int],
                    diversity_weight: float,
                    n_samples: int):
    return generator.sample_with_weights(num_samples=n_samples, cfc_weight=1,
                                         gower_weight=1, avg_gower_weight=1,
                                         bonus_objectives_weights=np.array(objective_weights).reshape((1, 2)),
                                         diversity_weight=diversity_weight, include_dataset=False)


In [9]:
TEXT_TARGET = "A futuristic black cyberpunk-style road racing bicycle"

run_id = str(datetime.now().strftime('%m-%d--%H.%M.%S')) + "-template-" + TEXT_TARGET

optimizer = CombinedOptimizer(
    design_targets=DesignTargets(
        continuous_targets=[
            ContinuousTarget(distance_column_name(0), 0, 1),
            ContinuousTarget(distance_column_name(1), 0, 1),
        ]
    ),
    target_embeddings=[TextEmbeddingTarget(text_target=TEXT_TARGET),
                       ImageEmbeddingTarget(image_path=resource_path('mtb.png'))
                       ],
    extra_bonus_objectives=[]
)
optimizer.set_starting_design_by_index('1')
generator = optimizer.build_generator(validation_functions=COMBINED_VALIDATION_FUNCTIONS,
                                      pop_size=500,
                                      features_to_vary=[f for f
                                                        in optimizer.starting_dataset.get_combined().columns
                                                        if ('bottle' not in f)
                                                        ]
                                      )
generator.use_empty_repair(False)

run_dir = run_result_path(run_id)
os.makedirs(run_dir, exist_ok=False)

generator.generate(200, seed=45)


Found framed material columns ['Material=Steel', 'Material=Aluminum', 'Material=Titanium']
Mapping Material=Steel to MATERIAL OHCLASS: STEEL
Mapping Material=Aluminum to MATERIAL OHCLASS: ALUMINUM
Mapping Material=Titanium to MATERIAL OHCLASS: TITANIUM
Found framed material columns ['Material=Steel', 'Material=Aluminum', 'Material=Titanium']
Mapping Material=Steel to MATERIAL OHCLASS: STEEL
Mapping Material=Aluminum to MATERIAL OHCLASS: ALUMINUM
Mapping Material=Titanium to MATERIAL OHCLASS: TITANIUM
Dropping clips material columns ['MATERIAL OHCLASS: ALUMINIUM', 'MATERIAL OHCLASS: BAMBOO', 'MATERIAL OHCLASS: CARBON', 'MATERIAL OHCLASS: OTHER', 'MATERIAL OHCLASS: STEEL', 'MATERIAL OHCLASS: TITANIUM']
Found framed material columns ['Material=Steel', 'Material=Aluminum', 'Material=Titanium']
Mapping Material=Steel to MATERIAL OHCLASS: STEEL
Mapping Material=Aluminum to MATERIAL OHCLASS: ALUMINUM
Mapping Material=Titanium to MATERIAL OHCLASS: TITANIUM
Found framed material columns ['Mater

/home/yazan/Repositories/Personal/mcd-clip/src/mcd_clip/bike_rider_fit/fit_analysis/vectorizedangles.py:158: RuntimeWarning: invalid value encountered in arccos
  alpha_1 = np.arccos((x_1 ** 2 - UL_s - x_2 ** 2) / (-2 * UL * x_2))
/home/yazan/Repositories/Personal/mcd-clip/src/mcd_clip/bike_rider_fit/fit_analysis/demoanalysis.py:76: RuntimeWarning: invalid value encountered in arccos
  Tksp2 = np.arccos(
/home/yazan/Repositories/Personal/mcd-clip/src/mcd_clip/bike_rider_fit/fit_analysis/vectorizedangles.py:158: RuntimeWarning: invalid value encountered in arccos
  alpha_1 = np.arccos((x_1 ** 2 - UL_s - x_2 ** 2) / (-2 * UL * x_2))


Found framed material columns ['Material=Steel', 'Material=Aluminum', 'Material=Titanium']
Mapping Material=Steel to MATERIAL OHCLASS: STEEL
Mapping Material=Aluminum to MATERIAL OHCLASS: ALUMINUM
Mapping Material=Titanium to MATERIAL OHCLASS: TITANIUM


/home/yazan/Repositories/Personal/mcd-clip/src/mcd_clip/bike_rider_fit/fit_analysis/interfacepoints.py:52: RuntimeWarning: invalid value encountered in sqrt
  DTx = np.sqrt(bike[:, 0] ** 2 - DTy ** 2)
/home/yazan/Repositories/Personal/mcd-clip/src/mcd_clip/bike_rider_fit/fit_analysis/vectorizedangles.py:158: RuntimeWarning: invalid value encountered in arccos
  alpha_1 = np.arccos((x_1 ** 2 - UL_s - x_2 ** 2) / (-2 * UL * x_2))
/home/yazan/Repositories/Personal/mcd-clip/src/mcd_clip/bike_rider_fit/fit_analysis/vectorizedangles.py:210: RuntimeWarning: invalid value encountered in arccos
  tors_ang = np.arccos((TL ** 2 + sth_dist ** 2 - x_1) / (2 * TL * sth_dist))
/home/yazan/Repositories/Personal/mcd-clip/src/mcd_clip/bike_rider_fit/fit_analysis/vectorizedangles.py:216: RuntimeWarning: invalid value encountered in arccos
  armpit_to_wrist = np.arccos((TL ** 2 + x_1 - sth_dist ** 2) / (2 * TL * (x_1 ** 0.5)))
/home/yazan/Repositories/Personal/mcd-clip/src/mcd_clip/bike_rider_fit/fit_anal

Found framed material columns ['Material=Steel', 'Material=Aluminum', 'Material=Titanium']
Mapping Material=Steel to MATERIAL OHCLASS: STEEL
Mapping Material=Aluminum to MATERIAL OHCLASS: ALUMINUM
Mapping Material=Titanium to MATERIAL OHCLASS: TITANIUM
embedding_distance_1 | min 0.6899999976158142, max [1.0499999523162842],  average [0.7300000190734863]
embedding_distance_2 | min 0.07999999821186066, max [1.5199999809265137],  average [0.12999999523162842]
Mapping SSB_Include to choice with options (0, 1)
Mapping CSB_Include to choice with options (0, 1)
Mapped CS Length to Real with bounds (0.322943, 0.5569195)
Mapped BB Drop to Real with bounds (-0.0374885, 0.10553499999999999)
Mapped SS E to Real with bounds (0.015602500000000002, 0.223252)
Mapped ST Angle to Real with bounds (65.99998244937126, 78.99999375043305)
Mapped BB OD to Real with bounds (0.03484, 0.053875)
Mapped TT OD to Real with bounds (0.024204999999999997, 0.04801)


...

Found framed material columns ['Material=Steel', 'Material=Aluminum', 'Material=Titanium']
Mapping Material=Steel to MATERIAL OHCLASS: STEEL
Mapping Material=Aluminum to MATERIAL OHCLASS: ALUMINUM
Mapping Material=Titanium to MATERIAL OHCLASS: TITANIUM
embedding_distance_1 | min 0.6800000071525574, max [1.1799999475479126],  average [0.7200000286102295]
embedding_distance_2 | min 0.07999999821186066, max [1.6799999475479126],  average [0.14000000059604645]
Validation result: fraction invalid [0.092]
Validation result: fraction invalid [0.096]
Validation result: fraction invalid [0.074]
Validation result: fraction invalid [0.102]
Validation result: fraction invalid [0.0]
Validation result: fraction invalid [0.004]
   199 |    99000 |    500 |  0.000000E+00 |  0.000000E+00 |  0.0061532005 |             f


/home/yazan/Repositories/Personal/mcd-clip/venv/lib/python3.10/site-packages/pymoo/operators/crossover/sbx.py:42: RuntimeWarning: divide by zero encountered in power
  alpha = 2.0 - np.power(beta, -(eta + 1.0))
/home/yazan/Repositories/Personal/mcd-clip/venv/lib/python3.10/site-packages/pymoo/operators/crossover/sbx.py:47: RuntimeWarning: invalid value encountered in power
  betaq[mask] = np.power((rand * alpha), (1.0 / (eta + 1.0)))[mask]
/home/yazan/Repositories/Personal/mcd-clip/venv/lib/python3.10/site-packages/pymoo/operators/mutation/pm.py:45: RuntimeWarning: invalid value encountered in power
  d = np.power(val, mut_pow) - 1.0


Found framed material columns ['Material=Steel', 'Material=Aluminum', 'Material=Titanium']
Mapping Material=Steel to MATERIAL OHCLASS: STEEL
Mapping Material=Aluminum to MATERIAL OHCLASS: ALUMINUM
Mapping Material=Titanium to MATERIAL OHCLASS: TITANIUM
Found framed material columns ['Material=Steel', 'Material=Aluminum', 'Material=Titanium']
Mapping Material=Steel to MATERIAL OHCLASS: STEEL
Mapping Material=Aluminum to MATERIAL OHCLASS: ALUMINUM
Mapping Material=Titanium to MATERIAL OHCLASS: TITANIUM


/home/yazan/Repositories/Personal/mcd-clip/src/mcd_clip/bike_rider_fit/fit_analysis/vectorizedangles.py:158: RuntimeWarning: invalid value encountered in arccos
  alpha_1 = np.arccos((x_1 ** 2 - UL_s - x_2 ** 2) / (-2 * UL * x_2))
/home/yazan/Repositories/Personal/mcd-clip/src/mcd_clip/bike_rider_fit/fit_analysis/demoanalysis.py:76: RuntimeWarning: invalid value encountered in arccos
  Tksp2 = np.arccos(
/home/yazan/Repositories/Personal/mcd-clip/src/mcd_clip/bike_rider_fit/fit_analysis/vectorizedangles.py:158: RuntimeWarning: invalid value encountered in arccos
  alpha_1 = np.arccos((x_1 ** 2 - UL_s - x_2 ** 2) / (-2 * UL * x_2))


Found framed material columns ['Material=Steel', 'Material=Aluminum', 'Material=Titanium']
Mapping Material=Steel to MATERIAL OHCLASS: STEEL
Mapping Material=Aluminum to MATERIAL OHCLASS: ALUMINUM
Mapping Material=Titanium to MATERIAL OHCLASS: TITANIUM
embedding_distance_1 | min 0.6899999976158142, max [1.1799999475479126],  average [0.7200000286102295]
embedding_distance_2 | min 0.07999999821186066, max [1.6699999570846558],  average [0.15000000596046448]
Validation result: fraction invalid [0.088]
Validation result: fraction invalid [0.076]
Validation result: fraction invalid [0.084]
Validation result: fraction invalid [0.086]
Validation result: fraction invalid [0.0]
Validation result: fraction invalid [0.0]
   200 |    99500 |    500 |  0.000000E+00 |  0.000000E+00 |  0.0064774167 |             f


In [37]:
desired_weight = 1
distance_column_index = 1
objective_weights = [desired_weight * (1 - distance_column_index), desired_weight * distance_column_index]
as_many = generator.sample_with_weights(num_samples=100_000, cfc_weight=0,
                                        gower_weight=0, avg_gower_weight=0,
                                        diversity_weight=0,
                                        bonus_objectives_weights=np.array(objective_weights).reshape((1, 2)),
                                        include_dataset=False)
column_ = distance_column_name(distance_column_index)
as_many[column_] = optimizer.predict(CombinedDataset(as_many))[column_]
closes_cfs = as_many.sort_values(by=column_, ascending=True)[:4]
print(f"Found closest cfs {closes_cfs[distance_column_name(distance_column_index)]}")


Only found 71316 valid counterfactuals! Returning all 71316.
Done! Returning CFs
Found framed material columns ['Material=Steel', 'Material=Aluminum', 'Material=Titanium']
Mapping Material=Steel to MATERIAL OHCLASS: STEEL
Mapping Material=Aluminum to MATERIAL OHCLASS: ALUMINUM
Mapping Material=Titanium to MATERIAL OHCLASS: TITANIUM
Found framed material columns ['Material=Steel', 'Material=Aluminum', 'Material=Titanium']
Mapping Material=Steel to MATERIAL OHCLASS: STEEL
Mapping Material=Aluminum to MATERIAL OHCLASS: ALUMINUM
Mapping Material=Titanium to MATERIAL OHCLASS: TITANIUM


/home/yazan/Repositories/Personal/mcd-clip/src/mcd_clip/bike_rider_fit/fit_analysis/interfacepoints.py:52: RuntimeWarning: invalid value encountered in sqrt
  DTx = np.sqrt(bike[:, 0] ** 2 - DTy ** 2)
/home/yazan/Repositories/Personal/mcd-clip/src/mcd_clip/bike_rider_fit/fit_analysis/vectorizedangles.py:158: RuntimeWarning: invalid value encountered in arccos
  alpha_1 = np.arccos((x_1 ** 2 - UL_s - x_2 ** 2) / (-2 * UL * x_2))
/home/yazan/Repositories/Personal/mcd-clip/src/mcd_clip/bike_rider_fit/fit_analysis/demoanalysis.py:76: RuntimeWarning: invalid value encountered in arccos
  Tksp2 = np.arccos(


Found framed material columns ['Material=Steel', 'Material=Aluminum', 'Material=Titanium']
Mapping Material=Steel to MATERIAL OHCLASS: STEEL
Mapping Material=Aluminum to MATERIAL OHCLASS: ALUMINUM
Mapping Material=Titanium to MATERIAL OHCLASS: TITANIUM


/home/yazan/Repositories/Personal/mcd-clip/src/mcd_clip/bike_rider_fit/fit_analysis/interfacepoints.py:52: RuntimeWarning: invalid value encountered in sqrt
  DTx = np.sqrt(bike[:, 0] ** 2 - DTy ** 2)
/home/yazan/Repositories/Personal/mcd-clip/src/mcd_clip/bike_rider_fit/fit_analysis/vectorizedangles.py:158: RuntimeWarning: invalid value encountered in arccos
  alpha_1 = np.arccos((x_1 ** 2 - UL_s - x_2 ** 2) / (-2 * UL * x_2))


embedding_distance_1 | min 0.6800000071525574, max [0.9900000095367432],  average [0.7200000286102295]
embedding_distance_2 | min 0.07000000029802322, max [0.9900000095367432],  average [0.12999999523162842]
Found closest cfs 52901    0.073831
44014    0.075459
60738    0.075982
53597    0.076036
Name: embedding_distance_2, dtype: float32


In [38]:
render_some(closes_cfs, 'latest-run', 1, 'both')

Found framed material columns ['Material=Steel', 'Material=Aluminum', 'Material=Titanium']
Mapping Material=Steel to MATERIAL OHCLASS: STEEL
Mapping Material=Aluminum to MATERIAL OHCLASS: ALUMINUM
Mapping Material=Titanium to MATERIAL OHCLASS: TITANIUM
Deleting key MATERIAL OHCLASS: ALUMINIUM
Deleting key MATERIAL OHCLASS: BAMBOO
Deleting key MATERIAL OHCLASS: CARBON
Deleting key MATERIAL OHCLASS: OTHER
Deleting key MATERIAL OHCLASS: STEEL
Deleting key MATERIAL OHCLASS: TITANIUM
Deleting key Dropout spacing style OHCLASS: 0
Deleting key Dropout spacing style OHCLASS: 1
Deleting key Dropout spacing style OHCLASS: 2
Deleting key Dropout spacing style OHCLASS: 3
Deleting key Fork type OHCLASS: 0
Deleting key Fork type OHCLASS: 1
Deleting key Fork type OHCLASS: 2
Deleting key Stem kind OHCLASS: 0
Deleting key Stem kind OHCLASS: 1
Deleting key Stem kind OHCLASS: 2
Deleting key Handlebar style OHCLASS: 0
Deleting key Handlebar style OHCLASS: 1
Deleting key Handlebar style OHCLASS: 2
Deleting